In [1]:
import os
import requests
import csv
from time import sleep

In [2]:
import requests
import time
import json

base_url = "https://dackkms.gov.in/Account/API/kKMS_QueryData.aspx"

def test_api(state, district, month, year):
    state_code = f"{state:02d}"
    district_code = f"{state_code}{district:02d}"
    params = {
        "StateCD": state_code,
        "DistrictCd": district_code,
        "Month": month,
        "Year": year
    }
    try:
        response = requests.get(base_url, params=params)
        return response.status_code, response.json()
    except requests.RequestException as e:
        return 0, str(e)

def extract_info(content):
    if content['ResponseCode'] == '1' and content['Response'] == 'Data Found':
        if content['data']:
            return content['data'][0]['StateName'], content['data'][0]['DistrictName']
    return None, None

def save_intermediate_results(states, districts, last_processed_state):
    with open(f'states_intermediate_{last_processed_state}.json', 'w') as f:
        json.dump(states, f, indent=2)
    with open(f'districts_intermediate_{last_processed_state}.json', 'w') as f:
        json.dump(districts, f, indent=2)
    print(f"\nSaved intermediate results up to state {last_processed_state}")

def main():
    states = {}
    districts = {}
    total_requests = 0
    successful_extractions = 0

    print("Starting the discovery process...")

    for state in range(1, 100):  # Checking state codes from 01 to 99
        state_code = f"{state:02d}"
        districts[state_code] = {}
        print(f"\nChecking state code: {state_code}")

        for district in range(1, 100):  # Assuming max 99 districts per state
            total_requests += 1
            status, content = test_api(state, district, 1, 2023)  # Using January 2023
            
            if status == 200 and isinstance(content, dict):
                state_name, district_name = extract_info(content)
                
                if state_name and district_name:
                    successful_extractions += 1
                    if state_code not in states:
                        states[state_code] = state_name
                        print(f"Discovered new state: {state_name} ({state_code})")

                    district_code = f"{state_code}{district:02d}"
                    districts[state_code][district_code] = district_name

                    print(f"Found: State {state_name} ({state_code}), District {district_name} ({district_code})")
                else:
                    print(f"No data for state code {state_code}, district number {district:02d}")
            else:
                print(f"Failed or empty response for state code {state_code}, district number {district:02d}")

            time.sleep(0.5)  # Add a small delay to avoid overwhelming the server

            # Print progress every 10 requests
            if total_requests % 10 == 0:
                print(f"Progress: {total_requests} requests made, {successful_extractions} successful extractions")

        # Save intermediate results every 3 states
        if state % 3 == 0:
            save_intermediate_results(states, districts, state)

        # Add a 15-second break after processing each state
        print(f"Finished processing state code {state_code}. Taking a 15-second break...")
        time.sleep(15)

    # Save the final results
    with open('states_final.json', 'w') as f:
        json.dump(states, f, indent=2)
    print("\nSaved final states data to states_final.json")

    with open('districts_final.json', 'w') as f:
        json.dump(districts, f, indent=2)
    print("Saved final districts data to districts_final.json")

    print(f"\nDiscovery process completed.")
    print(f"Total requests made: {total_requests}")
    print(f"Successful extractions: {successful_extractions}")
    print(f"States discovered: {len(states)}")
    print(f"Districts discovered: {sum(len(districts[state]) for state in districts)}")

if __name__ == "__main__":
    main()

Starting the discovery process...

Checking state code: 01
Discovered new state: ANDHRA PRADESH (01)
Found: State ANDHRA PRADESH (01), District SRIKAKULAM (0101)
Found: State ANDHRA PRADESH (01), District VIZIANAGARM (0102)
Found: State ANDHRA PRADESH (01), District VISAKHAPATNAM (0103)
Found: State ANDHRA PRADESH (01), District EAST GODAVARI (0104)
Found: State ANDHRA PRADESH (01), District WEST GODAVARI (0105)
Found: State ANDHRA PRADESH (01), District KRISHNA (0106)
Found: State ANDHRA PRADESH (01), District GUNTUR (0107)
Found: State ANDHRA PRADESH (01), District PRAKASAM (0108)
Found: State ANDHRA PRADESH (01), District NELLORE (0109)
Found: State ANDHRA PRADESH (01), District KURNOOL (0110)
Progress: 10 requests made, 10 successful extractions
Found: State ANDHRA PRADESH (01), District ANANTPUR (0111)
Found: State ANDHRA PRADESH (01), District Y S R  (0112)
Found: State ANDHRA PRADESH (01), District CHITTOOR (0113)
No data for state code 01, district number 14
No data for state c

KeyboardInterrupt: 

In [4]:
import requests
import time
import json
import os

base_url = "https://dackkms.gov.in/Account/API/kKMS_QueryData.aspx"

def test_api(state, district, month, year):
    state_code = f"{state:02d}"
    district_code = f"{state_code}{district:02d}"
    params = {
        "StateCD": state_code,
        "DistrictCd": district_code,
        "Month": month,
        "Year": year
    }
    try:
        response = requests.get(base_url, params=params)
        return response.status_code, response.json()
    except requests.RequestException as e:
        return 0, str(e)

def extract_info(content):
    if content['ResponseCode'] == '1' and content['Response'] == 'Data Found':
        if content['data']:
            return content['data'][0]['StateName'], content['data'][0]['DistrictName']
    return None, None

def save_intermediate_results(states, districts, last_processed_state):
    with open(f'states_intermediate_{last_processed_state}.json', 'w') as f:
        json.dump(states, f, indent=2)
    with open(f'districts_intermediate_{last_processed_state}.json', 'w') as f:
        json.dump(districts, f, indent=2)
    print(f"\nSaved intermediate results up to state {last_processed_state}")

def load_intermediate_results(last_processed_state):
    with open(f'states_intermediate_{last_processed_state}.json', 'r') as f:
        states = json.load(f)
    with open(f'districts_intermediate_{last_processed_state}.json', 'r') as f:
        districts = json.load(f)
    return states, districts

def find_last_processed_state():
    files = os.listdir()
    intermediate_files = [f for f in files if f.startswith('states_intermediate_') and f.endswith('.json')]
    if not intermediate_files:
        return 0
    last_file = max(intermediate_files)
    return int(last_file.split('_')[-1].split('.')[0])

def main(start_state=None):
    if start_state is None:
        last_processed_state = find_last_processed_state()
        start_state = last_processed_state + 1
        if last_processed_state > 0:
            print(f"Resuming from state {start_state}")
            states, districts = load_intermediate_results(last_processed_state)
        else:
            print("Starting new discovery process")
            states, districts = {}, {}
    else:
        print(f"Starting from specified state {start_state}")
        states, districts = {}, {}

    total_requests = 0
    successful_extractions = 0

    for state in range(start_state, 100):  # Checking state codes from start_state to 99
        state_code = f"{state:02d}"
        if state_code not in districts:
            districts[state_code] = {}
        print(f"\nChecking state code: {state_code}")

        for district in range(1, 100):  # Assuming max 99 districts per state
            total_requests += 1
            status, content = test_api(state, district, 1, 2023)  # Using January 2023
            
            if status == 200 and isinstance(content, dict):
                state_name, district_name = extract_info(content)
                
                if state_name and district_name:
                    successful_extractions += 1
                    if state_code not in states:
                        states[state_code] = state_name
                        print(f"Discovered new state: {state_name} ({state_code})")

                    district_code = f"{state_code}{district:02d}"
                    districts[state_code][district_code] = district_name

                    print(f"Found: State {state_name} ({state_code}), District {district_name} ({district_code})")
                else:
                    print(f"No data for state code {state_code}, district number {district:02d}")
            else:
                print(f"Failed or empty response for state code {state_code}, district number {district:02d}")

            time.sleep(0.5)  # Add a small delay to avoid overwhelming the server

            # Print progress every 10 requests
            if total_requests % 10 == 0:
                print(f"Progress: {total_requests} requests made, {successful_extractions} successful extractions")

        # Save intermediate results every 3 states
        if state % 3 == 0:
            save_intermediate_results(states, districts, state)

        # Add a 15-second break after processing each state
        print(f"Finished processing state code {state_code}. Taking a 15-second break...")
        time.sleep(15)

    # Save the final results
    with open('states_final.json', 'w') as f:
        json.dump(states, f, indent=2)
    print("\nSaved final states data to states_final.json")

    with open('districts_final.json', 'w') as f:
        json.dump(districts, f, indent=2)
    print("Saved final districts data to districts_final.json")

    print(f"\nDiscovery process completed.")
    print(f"Total requests made: {total_requests}")
    print(f"Successful extractions: {successful_extractions}")
    print(f"States discovered: {len(states)}")
    print(f"Districts discovered: {sum(len(districts[state]) for state in districts)}")

if __name__ == "__main__":
    main(start_state=16)

Starting from specified state 16

Checking state code: 16
Discovered new state: PUNJAB (16)
Found: State PUNJAB (16), District HOSHIARPUR (1601)
Found: State PUNJAB (16), District JALANDHAR (1602)
Found: State PUNJAB (16), District LUDHIANA (1603)
Found: State PUNJAB (16), District FEROZPUR (1604)
Found: State PUNJAB (16), District AMRITSAR (1605)
Found: State PUNJAB (16), District GURDASPUR (1606)
Found: State PUNJAB (16), District KAPURTHALA (1607)
Found: State PUNJAB (16), District BHATINDA (1608)
Found: State PUNJAB (16), District PATIALA (1609)
Found: State PUNJAB (16), District SANGRUR (1610)
Progress: 10 requests made, 10 successful extractions
Found: State PUNJAB (16), District RUPNAGAR (1611)
Found: State PUNJAB (16), District FARIDKOT (1612)
Found: State PUNJAB (16), District MANSA (1613)
Found: State PUNJAB (16), District FATEHGARH SAHIB (1614)
Found: State PUNJAB (16), District SAHIBZADA AJIT SINGH NAGAR (1615)
Found: State PUNJAB (16), District MOGA (1616)
Found: State PUN

KeyboardInterrupt: 